In [2]:
import os

In [3]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/home/himanshu/github_himanshu/end-to-end-ML-with-MLflow'

In [6]:
from dataclasses import dataclass  # Import the dataclass decorator for creating data classes
from pathlib import Path  # Import the Path class for handling filesystem paths

@dataclass(frozen=True)  # Define the class as a dataclass and make it immutable with frozen=True
class DataTransformationConfig:
    """
    Configuration class for data transformation.

    Attributes:
        root_dir (Path): The root directory where data transformation operations will take place.
        data_path (Path): The path to the data that will be transformed.
    """
    root_dir: Path
    data_path: Path


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [9]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd 

[2024-06-13 13:15:48,954: INFO: utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-06-13 13:15:48,955: INFO: utils: NumExpr defaulting to 8 threads.]


In [10]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """
        Initialize ConfigurationManager with paths to the configuration files.
        
        Args:
            config_filepath (Path): Path to the configuration YAML file.
            params_filepath (Path): Path to the parameters YAML file.
            schema_filepath (Path): Path to the schema YAML file.
        """
        self.config = read_yaml(config_filepath)  # Read the configuration file
        self.params = read_yaml(params_filepath)  # Read the parameters file
        self.schema = read_yaml(schema_filepath)  # Read the schema file

        create_directories([self.config.artifacts_root])  # Create the root directory for artifacts

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """
        Get the DataTransformationConfig object from the configuration.
        
        Returns:
            DataTransformationConfig: Configuration for data transformation.
        """
        config = self.config.data_transformation  # Get the data transformation configuration section

        create_directories([config.root_dir])  # Ensure the root directory for data transformation exists

        # Create and return a DataTransformationConfig object with the configuration details
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config


In [11]:
try:
    # Initialize ConfigurationManager with default file paths
    config = ConfigurationManager()
    
    # Retrieve data transformation configuration
    data_transformation_config = config.get_data_transformation_config()
    
    # Initialize DataTransformation with the retrieved configuration
    data_transformation = DataTransformation(config=data_transformation_config)
    
    # Perform train-test splitting
    data_transformation.train_test_spliting()
except Exception as e:
    # Raise the exception if any error occurs
    raise e


[2024-06-13 13:17:58,594: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-06-13 13:17:58,597: INFO: common: YAML file: params.yaml loaded successfully]
[2024-06-13 13:17:58,600: INFO: common: YAML file: schema.yaml loaded successfully]
[2024-06-13 13:17:58,601: INFO: common: Created directory at: artifacts]
[2024-06-13 13:17:58,602: INFO: common: Created directory at: artifacts/data_transformation]
[2024-06-13 13:17:58,633: INFO: 602995403: Splited data into training and test sets]
[2024-06-13 13:17:58,635: INFO: 602995403: (1199, 12)]
[2024-06-13 13:17:58,636: INFO: 602995403: (400, 12)]
(1199, 12)
(400, 12)
